In [1]:
BASE_ELASTIC_URL = "http://192.168.36.98/mytimes/elasticCommentQuery?sort=desc&appKey=TOI,ET&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED"
BATCH_SIZE_PARAM = 250
# ITER_BUFFER = 2
CSV_FILE_NAME = '/Users/yash.dalmia/elasticComments.csv'
DEFAULT_FIELDS = ['C_T']

#     field_arr = ['c_id', 'C_T']
#     field_arr = ['c_id', 'msid', 'C_T', 'C_D_Minute', 'ED_ID', 'C_APP_RES']
    
#     field_arr = ['C_T']
#     csv_list = [[(x[fieldStr] if (fieldStr in x) else "") for fieldStr in field_arr] for x in list_comments if ('C_T' in x)]
#     field_arr = ['c_id', 'msid', 'C_T', 'ERO_D', 'F_NAME', 'F_ADD', 'C_D', 'A_ID', 'C_APP_RES']

In [2]:
startEpoch = None
endEpoch = None

In [3]:
import urllib.request, json 
import math
import csv
import pandas as pd

In [4]:
def getJsonFromUrl(url_param):
    with urllib.request.urlopen(url_param) as url:
        data = json.loads(url.read().decode())
        return data

In [5]:
def createNewCsv(file_path = CSV_FILE_NAME):
    with open(file_path, 'w+') as writeFile:
        pass

In [6]:
def getCsvListFromListComments(list_comments, fields_to_take = DEFAULT_FIELDS):
    
    csv_list = []
    for x in list_comments:
        if 'C_T' in x:
            list1 = []
            for fieldStr in fields_to_take:
                if "." in fieldStr:
                    fieldStrArr = fieldStr.split(".")
                    if fieldStrArr[0] in x and fieldStrArr[1] in x[fieldStrArr[0]]:
                        list1.append(x[fieldStrArr[0]][fieldStrArr[1]])
                    else:
                        list1.append("")
                elif fieldStr in x:
                    list1.append(x[fieldStr])
                else:
                    list1.append("")
            csv_list.append(list1)
            
    return csv_list

In [7]:
def getModifiedUrl(url, from_param, size_param, startEpoch=None, endEpoch=None):
    result_url = url + "&from=" + str(from_param) + "&size=" + str(size_param)
    if (startEpoch!=None and endEpoch!=None):
        result_url = result_url + "&sDateEpoch=" + str(startEpoch) + "&eDateEpoch=" + str(endEpoch)
    return result_url

In [8]:
def getModifiedUrlDate(url, startEpoch=None, endEpoch=None):
#     result_url = url + "&from=" + str(from_param) + "&size=" + str(size_param)
    result_url = url
    if (startEpoch!=None and endEpoch!=None):
        result_url = result_url + "&sDateEpoch=" + str(startEpoch) + "&eDateEpoch=" + str(endEpoch)
    return result_url

In [9]:
def appendUrlResultToCsv(url, fields_to_take = DEFAULT_FIELDS, size_param = BATCH_SIZE_PARAM, file_path = CSV_FILE_NAME, force = False, to_download_till = None):
    
    final_url = getModifiedUrl(url, 0, size_param)
    json_raw_response = getJsonFromUrl(final_url)

    total_comment_count = json_raw_response['hits']['total']
    
    ELASTIC_LIMIT = 10000
    if (to_download_till != None and to_download_till < total_comment_count):
        total_comment_count = to_download_till
        
    if (total_comment_count > ELASTIC_LIMIT):
        print("total_comment_count greater than 10000: %d" %(total_comment_count))
        if ~force:
            print("aborting")
            return
        else:
            total_comment_count = ELASTIC_LIMIT
        
    
    num_iters = math.ceil(total_comment_count/size_param)
#     num_iters = num_iters + ITER_BUFFER

    print("total_count : %d" %(total_comment_count))
    print("num_iters : %d" %(num_iters))

    for iter_val in range(0, num_iters, 1):
        print("iteration : %d" %(iter_val))
        from_val = iter_val * size_param
        final_url = getModifiedUrl(url, from_val, size_param)
        json_raw_response = getJsonFromUrl(final_url)
        list_comments = [x['_source'] for x in json_raw_response['hits']['hits']]
        
        csv_list = getCsvListFromListComments(list_comments, fields_to_take)
        df_csv = pd.DataFrame(csv_list, columns = fields_to_take)
        
        with open(file_path, 'a') as f:
            df_csv.to_csv(f, index = False, header=f.tell()==0, encoding='utf-8')

In [ ]:
# base_url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&appKey=TOI&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED"
base_url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&appKey=TOI&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED"

csv_file_path = "data/TOI_1_June.csv"
fields_to_take = ['C_T']

batch_write_size = 250

createNewCsv(file_path = csv_file_path)

NET_END_TIME_EPOCH = 1559413800000
NUM_DAYS_IN_ONE_ITER = 0.25
NUM_ITERS = 1*4

# endEpoch = NET_START_TIME_EPOCH
startEpoch = NET_END_TIME_EPOCH

for iter_val in range(0, NUM_ITERS, 1):
    print("\nIter_val : %d" %(iter_val))
    endEpoch = startEpoch
    startEpoch = endEpoch - int(NUM_DAYS_IN_ONE_ITER*24*60*60*1000)
    
    url = getModifiedUrlDate(base_url, startEpoch, endEpoch)
    print("url : " + url)
    appendUrlResultToCsv(url, fields_to_take = fields_to_take, size_param = batch_write_size, file_path = csv_file_path)

print("\nnet startEpoch : ", startEpoch)
print("net endEpoch : ", NET_END_TIME_EPOCH)
print("csv written to %s"%(csv_file_path))

# url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&appKey=NBTO&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&sDate=24%2F4%2F2019&eDate=24%2F4%2F2019"
url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&sDateEpoch=1548412448000&eDateEpoch=1556188448000&appKey=TOI&queryString=_exists_:SPAM_VAL%20AND%20SPAM_VAL.isSpam:false&multiActorId=6179270,665245713,14219596,6267354,665046059,549930791,548178454,668244979,663707220,22541448,2366863,2589915,655718924,526173678,659669241,103473849,2515704"
from_param = 0
size_param = 50
# print(getModifiedUrl(url, from_param, size_param))
csv_file_path = "data/most_abusive_TOI_last_90_days_25April.csv"
fields_to_take = ['c_id', 'C_T', 'SPAM_VAL.round_prob']

createNewCsv(file_path = csv_file_path)

appendUrlResultToCsv(url, fields_to_take = fields_to_take, size_param = 250, file_path = csv_file_path)
print('saved file at {}'.format(csv_file_path))

In [10]:
# csv_file_path = "/Users/yash.dalmia/NBT_spam_13-19June.csv"
# createNewCsv(file_path = csv_file_path)

In [11]:
# base_url = "http://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&appKey=NBTO&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&sDate=13%2F6%2F2019&rejectionType=ML_PROCESS"
# fields_to_take = ['C_T', 'C_D']
# batch_write_size = 150
# appendUrlResultToCsv(base_url, fields_to_take = fields_to_take, size_param = batch_write_size, file_path = csv_file_path)

total_count : 71
num_iters : 1
iteration : 0


In [15]:
# import time
# get_time = lambda x : time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x/1000))

In [12]:
# df_r = pd.read_csv(csv_file_path)

In [18]:
# df_r['time'] = df_r['C_D'].apply(get_time)

In [20]:
# get_time(1560924551196)

In [22]:
# df_r = df_r.drop(columns=['C_D'])

In [28]:
# df_r.rename(index=str, columns={'C_T': 'Comment Text', 'time': 'Time'}, inplace=True)

In [30]:
# df_r.to_csv(csv_file_path, index=False, encoding='utf-8')